In [1]:
%config IPCompleted.greedy = True

# Import Libraries

In [2]:
import os
from azureml.core import Dataset, Environment, Experiment, ScriptRunConfig, Workspace
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.widgets import RunDetails
from pathlib import Path

# Get Workspace

In [3]:
ws = Workspace.from_config()
ws.get_details()

{'id': '/subscriptions/1f3811a0-1fb9-4d43-974c-8c7bdc025d92/resourceGroups/disasterLocator/providers/Microsoft.MachineLearningServices/workspaces/disasterLocatorML',
 'name': 'disasterLocatorML',
 'location': 'eastus',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'sku': 'Basic',
 'workspaceid': '90beb71e-3575-4cbd-af2a-cb2c303edc7b',
 'description': '',
 'friendlyName': 'disasterLocatorML',
 'creationTime': '2020-11-11T14:39:30.8601986+00:00',
 'containerRegistry': '/subscriptions/1f3811a0-1fb9-4d43-974c-8c7bdc025d92/resourceGroups/disasterLocator/providers/Microsoft.ContainerRegistry/registries/90beb71e35754cbdaf2acb2c303edc7b',
 'keyVault': '/subscriptions/1f3811a0-1fb9-4d43-974c-8c7bdc025d92/resourcegroups/disasterlocator/providers/microsoft.keyvault/vaults/disasterlocato9806975694',
 'applicationInsights': '/subscriptions/1f3811a0-1fb9-4d43-974c-8c7bdc025d92/resourcegroups/disasterlocator/providers/microsoft.insights/components/disasterlocato5776021600',
 

# Attach Compute Target

In [5]:
# compute cluster configuration
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "nlp-cpu-cluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target: ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                                min_nodes=compute_min_nodes,
                                                                max_nodes=compute_max_nodes)
    # create the cluster
    compute_target = ComputeTarget.create(
        ws, compute_name, provisioning_config)

    # can poll for a minimum number of nodes and for a specific timeout.
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(
        show_output=True, min_node_count=None, timeout_in_minutes=20)

    # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

found compute target: nlp-cpu-cluster


# Create Experiment

In [6]:
experiment_name = 'cnn-1convolutional-layer'
experiment = Experiment(workspace=ws, name=experiment_name)

# Define Environment

In [7]:
keras_env = Environment.from_conda_specification(name='keras-env', file_path='../conda_dependencies.yml')

## Create a ScriptRunConfig

In [8]:
script_folder = os.path.join(Path(os.getcwd()).resolve().parent, 'src')
script_folder

'C:\\Users\\luisf\\github\\DisastersLocator\\src'

In [9]:
args = [
        '--batch-size', 16,
       '--epochs', 2]

src = ScriptRunConfig(source_directory=script_folder,
                      script='train_cnn.py',
                      arguments=args,
                      compute_target=compute_target,
                      environment=keras_env)

src.run_config.source_directory_data_store = "workspaceblobstore" 

# Submit Run

In [10]:
run = experiment.submit(src)
run

Experiment,Id,Type,Status,Details Page,Docs Page
cnn-1convolutional-layer,cnn-1convolutional-layer_1605384155_f029d818,azureml.scriptrun,Queued,Link to Azure Machine Learning studio,Link to Documentation


In [11]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [12]:
run.wait_for_completion(show_output=False)
run.get_metrics()

{'accuracy': 0.7756813417190775}

# Register the Model

In [13]:
model = run.register_model(model_name = "disaster_predictor_cnn", model_path = "outputs/model")